### Automation Script Overhaul 
For automating the parsing, transformation, and geographic file creation from .GPX<br><br>
Work taken from the script by Isias (ler_gpx.py) and workflow by Simone. Further developments by Kyle & Isais. <br><br>
Data collection must be done with Locus Map 4.x or formatted similarly to GPX files exportd by Locus Map 4 for this script to work.

### Section 1
This section takes the raw GPX files and makes them ready for QGIS and statistical analysis.

##### Daily Data
This next for loop / cell will create files with the data sorted by day and must be run before creaating data files by scan.

In [1]:
import gpxpy
import pandas as pd
import geopandas as gpd
import warnings
import os
from os import mkdir
from datetime import datetime, timedelta
from shapely import wkt
from shapely.geometry import LineString


import spatialFunctions
from spatialFunctions import *

In [2]:
dir_sel = ''
if dir_sel:
    mastercsv = pd.read_csv(dir_sel+'/csvDayFiles/scansMaster.csv',)
else:
    mastercsv = pd.read_csv('csvDayFiles/scansMaster.csv',)

mastercsv['centroid'] = mastercsv['centroid'].apply(wkt.loads)
master = gpd.GeoDataFrame(mastercsv, geometry='centroid', crs='EPSG:31985')
master.head()

,date,scan,zone,area(m2),perimeter(m),individuals,ind/ha,ind/perim(km),centroid,centBorder(m)
0,20220620,1,"POLYGON ((283145.4720175263 9168160.514930412,...",8609.506948,388.503749,19,2.206863,48.905577,POINT (283206.562 9168199.737),40.886908
1,20220620,2,"POLYGON ((283184.148496979 9168102.394768221, ...",1451.695042,192.834308,6,4.133099,31.114795,POINT (283190.636 9168128.658),113.217419
2,20220620,3,POLYGON ((283213.29180710355 9168225.197127782...,1959.275747,244.836270,30,15.311780,122.530865,POINT (283170.159 9168270.322),6.677663
3,20220620,4,POLYGON ((283171.64118206466 9168257.309558447...,940.278005,229.421717,12,12.762183,52.305423,POINT (283121.757 9168318.136),11.565621
4,20220620,5,POLYGON ((283133.48273913073 9168200.061091943...,2352.304260,198.399968,20,8.502301,100.806468,POINT (283137.868 9168236.302),53.358743


In [3]:
dfmaster = master[master['scan'].apply(lambda x: str(x).isdigit())]
dfmaster = dfmaster.sort_values('date').reset_index(drop=True)
dfmaster.head()

,date,scan,zone,area(m2),perimeter(m),individuals,ind/ha,ind/perim(km),centroid,centBorder(m)
0,20220619,1,POLYGON ((283088.25231338013 9168341.002126466...,6627.336846,391.314825,11,1.659792,28.110358,POINT (283045.385 9168416.038),60.456421
1,20220619,2,POLYGON ((283106.25612520013 9168387.872948287...,978.396110,161.174806,17,17.375376,105.475542,POINT (283100.779 9168423.160),5.181866
2,20220619,3,"POLYGON ((283108.6540333381 9168320.96190676, ...",731.572121,155.182701,17,23.237627,109.548293,POINT (283106.408 9168367.137),13.982538
3,20220619,4,POLYGON ((283203.53254056745 9168235.219232237...,6253.122656,460.821130,18,2.878562,39.060709,POINT (283158.620 9168272.627),13.891957
4,20220620,1,"POLYGON ((283145.4720175263 9168160.514930412,...",8609.506948,388.503749,19,2.206863,48.905577,POINT (283206.562 9168199.737),40.886908


In [4]:
groupmaster = dfmaster.groupby(['date']).agg({'centroid':list})
groupmaster['centroid'] = groupmaster['centroid'].apply(lambda x: LineString(x))
groupedmaster = gpd.GeoDataFrame(groupmaster)
groupedmaster.to_csv('temp.csv')
groupedmaster.head()

,centroid
date,
20220619,LINESTRING (283045.3850421915 9168416.03754092...
20220620,LINESTRING (283206.56196952634 9168199.7371894...


In [5]:
lines = pd.read_csv('temp.csv',)
lines.rename(columns={'centroid':'geometry'}, inplace=True)
lines['geometry'] = lines['geometry'].apply(wkt.loads)
lines = gpd.GeoDataFrame(lines, geometry='geometry', crs='EPSG:31985')
# lines = lines.reset_index(drop=True)
lines.head()

,date,geometry
0,20220619,"LINESTRING (283045.385 9168416.038, 283100.779..."
1,20220620,"LINESTRING (283206.562 9168199.737, 283190.636..."


In [6]:
for i, row in lines.iterrows():
    date = str(row['date'])
    dateline = lines.loc[[i]]
    dateline.loc[:,'length'] = dateline.length

    if dir_sel:
        dateline.to_file(dir_sel+'/gpkgData/'+date+'scans.gpkg', driver="GPKG", layer=date+'_route')
    else:
        dateline.to_file('gpkgData/'+date+'scans.gpkg', driver="GPKG", layer=date+'_route')

/home/kyle/Nextcloud/Monkey_Research/Data_Work/capuchin_geoanalysis/venvScans/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/kyle/Nextcloud/Monkey_Research/Data_Work/capuchin_geoanalysis/venvScans/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [14]:
scanCheck = dfmaster['scan'].shift(-1)

dfmaster.loc[:,'scanCheck'] = scanCheck

dfmaster['scan'] = dfmaster['scan'].astype(str).astype(int)
print(dfmaster.dtypes)

if dfmaster['scan'].astype({'scan':int}) == dfmaster['scanCheck']-1:
    point2 = dfmaster['centroid'].shift(-1)
point2 = gpd.GeoDataFrame(point2, geometry='centroid', crs='EPSG:31985')

dfmaster = gpd.GeoDataFrame(dfmaster, geometry='centroid', crs='EPSG:31985')
dfmaster['distCenCen(m)'] = dfmaster['centroid'].distance(point2)

dfmaster

date                int64
scan                int64
zone               object
area(m2)          float64
perimeter(m)      float64
individuals         int64
ind/ha            float64
ind/perim(km)     float64
centroid         geometry
centBorder(m)     float64
scanCheck         float64
dtype: object


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [27]:
mastercsv = mastercsv.sort_values('date').reset_index(drop=True)
shiftPos = mastercsv.pop('zone')
mastercsv['zone'] = shiftPos
shiftPos = mastercsv.pop('centroid')
mastercsv['centroid'] = shiftPos

In [ ]:
# remove last scan of each day dist


In [28]:
if dir_sel:
    mastercsv.to_csv(dir_sel+'/csvDayFiles/scansMaster1.csv')
else:
    mastercsv.to_csv('csvDayFiles/scansMaster1.csv')

os.remove('temp.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'temp.csv'

In [1]:


# Remove warning message for future warnings / Remover mensagem de aviso para avisos futuros
warnings.filterwarnings(action='ignore',category=FutureWarning)
# Suppress warning for setting with copy, non-issue here / Suprimir aviso para configuração com cópia, não é problema aqui
pd.set_option('mode.chained_assignment',None)

# Get user decision for input fields and state the variables / Obtenha a decisão do usuário para campos de entrada e indique as variáveis
userInput = input('To add observer, group, and weather information for each day type \'yes\'. Otherwise leave blank and hit enter')
observer = ''
group = ''
weather = ''

# Create a dictionary with all gpx files in dictionary / Crie um dicionário com todos os arquivos gpx no dicionário

gpxDict = dict()

for file in os.listdir():
    if file.endswith('.gpx'):
       gpxDict[file] = 'file_'+file

# Run for loop to cover every gpx file in directory / Execute o loop para cobrir todos os arquivos gpx no diretório

for i in gpxDict:
    
    # Open and read in the .gpx to a dataframe / Abra e leia no .gpx para um dataframe
    gpxCurrent = i
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    df = pd.read_xml(gpxCurrent)

    # Ask for observer, group, climate conditions / Pergunte por observador, grupo, condições climáticas
    if userInput:
        observer = input('Input for '+i+': Observer/Observador? ')
        group = input('Input for '+i+': Group/Grupo? (if both, mark 0) ') # or leave blank?
        weather = input('Input for '+i+': Weather conditions/Condição do clima? ')
    
    # Remove unecessary columns / Remova colunas desnecessárias
    df.pop('desc')
    df.pop('time')
    if 'hdop' in df.columns:
        df.pop('hdop')
    df = df.drop(index=0)

    # Reorganize columns / Reorganizar colunas
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)

    # Insert user input columns if they have a value / Insira colunas de entrada do usuário se elas tiverem um valor
    if observer:
        df.insert(loc=1, column='observer', value=observer, allow_duplicates=True)
    if group:
        df.insert(loc=1, column='group', value=group, allow_duplicates=True)
    if weather:
        df.insert(loc=1, column='weather', value=weather, allow_duplicates=True)

    # Split 'name' into date, time, and observations / Dividir 'nome' em data, hora e observações
    date = df['name'].str[:10]
    df.insert(loc=0, column='date', value=date, allow_duplicates=True)

    time = df['name'].str[11:19]
    df.insert(loc=1, column='time', value=time, allow_duplicates=True)

    obs = df['name'].str[19:]
    df.insert(loc=2, column='obs', value=obs, allow_duplicates=True)
    # Remove whitespace from observations column / Remover espaço em branco da coluna de observações
    df['obs'] = df['obs'].str.strip()

    df.pop('name')

    # Run the timeScan method in spatialFunctions to apply each point to its appropriate scan
    # Execute o método timeScan em spatialFunctions para aplicar cada ponto à sua varredura apropriada
    timeScan(df)
    
    # Run the observations method in spatialFunctions / Execute o método de observações em spatialFunctions
    observations(df)

    # Make geographic and set CRS / Faça geográfica e defina CRS
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat)) 
    gdf = gdf.set_crs('EPSG:4326')
    gdf = gdf.to_crs('EPSG:31985')

    # Check and create save directory for gpkg files / Verifique e crie um diretório de salvamento para arquivos gpkg
    gpkgsavePath = './gpkgData'
    isDir = os.path.isdir(gpkgsavePath)
    if isDir == False:
        mkdir('gpkgData')
    
    # Export gdf into gpkg / Exportar gdf para gpkg
    gdf.to_file('gpkgData/'+i[:-4]+'scans.gpkg', driver="GPKG", layer=i[:-4]+'_wholeDay')
    
    # Export each scan as a separate layer using the scanExport and scanSpatial methods in spatialFunctions
    # Exporte cada varredura como uma camada separada usando os métodos scanExport e scanSpatial em spatialFunctions    
    scanExport(gdf, i)

    # Check and create save directory for csv files / Verifique e crie um diretório de salvamento para arquivos csv
    csvsavePath = './csvDayFiles'
    isDir = os.path.isdir(csvsavePath)
    if isDir == False:
        mkdir('csvDayFiles')
    
    # Save to csv / Salvar em csv
    gdf.to_csv('csvDayFiles/'+i[:-4]+'.csv')

#### Section 2
Analysis of scans, analyze all of this for every individual scan

#### For GPKG data stored in gpkgData folder

In [ ]:
import pandas as pd
import geopandas as gpd

gpkgDict = dict()

for file in os.listdir('gpkgData'):
    if file.endswith('.gpkg'):
       gpkgDict[file] = 'file_'+file




# for i in gpkgDict:


In [2]:
areatest = gpd.read_file('gpkgData\/10062022scans.gpkg', layer = '10062022_scan1_zone')
areatest

,area,perimeter,individuals,ind/m2,ind/perim(m),geometry
0,1604.925306,198.37454,19,0.011839,0.095778,"POLYGON ((283114.650 9168313.246, 283083.739 9..."


In [ ]:
# Distance between each centroid in temporal order
# Map with lines and arrows for path throughout the day


In [ ]:
# Distance for each individual to the border

In [ ]:
# Export all relevant data to the main daily csv, i.e all dists etc for scans
# Create a new(?) csv for scan by scan data such as areas, perims, individuals, and derived data

In [ ]:
# transfer home range from the R script


In [ ]:
# Subgroups/cluster analysis, find clusters on eah scan and distance from each sub-centroid to main group centroid


#### Export Data

In [ ]:
# Export data frame to gpkg for use in QGIS / Exportar quadro de dados para gpkg para uso no QGIS
